In [1]:
import pandas as pd
import os as os
import numpy as np

#Files to load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

#Creation of Dataframes
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

In [2]:
#Checking for null values
school_data_df.count()

School ID      15
school_name    15
type           15
size           15
budget         15
dtype: int64

In [3]:
#Checking for null values
student_data_df.count()

Student ID       39170
student_name     39170
gender           39170
grade            39170
school_name      39170
reading_score    39170
math_score       39170
dtype: int64

In [4]:
# Determine if there are any missing values in the student data.
school_data_df.isnull().sum()

School ID      0
school_name    0
type           0
size           0
budget         0
dtype: int64

In [5]:
# Determine if there are any missing values in the student data.
student_data_df.isnull().sum()

Student ID       0
student_name     0
gender           0
grade            0
school_name      0
reading_score    0
math_score       0
dtype: int64

In [6]:
# Removing prefix and suffix from names
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")  

In [7]:
#Changing Thomas High School's 9th graders grade to NaN due to academic dishonesty
student_data_df.loc[(student_data_df.school_name == "Thomas High School") & (student_data_df.grade == "9th") ,"reading_score"]=np.nan
student_data_df.loc[(student_data_df.school_name == "Thomas High School") & (student_data_df.grade == "9th") ,"math_score"]=np.nan

In [8]:
# Combine the data into a single dataset.
school_data_complete_df = pd.merge(student_data_df, school_data_df, on=["school_name", "school_name"])

In [9]:
# Get the total number of students and schools
student_count = school_data_complete_df["Student ID"].count()
school_count = len(school_data_complete_df["school_name"].unique())

# Calculate the total budget given to schools
total_budget = school_data_df["budget"].sum()

# Calculate the average reading and math score of all the students
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

In [10]:
#Get passing percentage of math and reading per student (grade >=70)
passing_math = school_data_complete_df[school_data_complete_df["math_score"] >= 70]
passing_reading = school_data_complete_df[school_data_complete_df["reading_score"] >= 70]

# Calculate the number of students passing math and reading
passing_math_count = passing_math["student_name"].count()
passing_reading_count = passing_reading["student_name"].count()

In [11]:
# Calculate the percent that passed math and reading and both
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_reading_percentage = passing_reading_count / float(student_count) * 100

passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)]
overall_passing_math_reading_count = passing_math_reading["student_name"].count()
overall_passing_percentage = overall_passing_math_reading_count / float(student_count) * 100

In [12]:
# Creating District Summary DataFrame and formatting columns
district_summary_df = pd.DataFrame(
          [{"Total Schools": school_count,
          "Total Students": student_count,
          "Total Budget": total_budget,
          "Average Math Score": average_math_score,
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading_percentage,
        "% Overall Passing": overall_passing_percentage}])

district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.0f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.0f}%".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.0f}%".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.9,81.9,74%,85%,64%


In [13]:
# Determine the school type.
per_school_types = school_data_df.set_index(["school_name"])["type"]

In [14]:
# Calculate the total student count.
per_school_counts = school_data_complete_df["school_name"].value_counts()

# Calculate the total school budget.
per_school_budget = school_data_df.set_index(["school_name"])["budget"]

#Calculate total spending per student
per_school_capita = per_school_budget / per_school_counts

# Calculate the average math scores.
per_school_averages = school_data_complete_df.groupby(["school_name"]).mean()

# Calculate the average test scores.
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

# Calculate the % passing scores.
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_math = per_school_passing_math / per_school_counts * 100

per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

per_school_passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)]
per_school_passing_math_reading = per_school_passing_math_reading.groupby(["school_name"]).count()["student_name"]
per_overall_passing_percentage = per_school_passing_math_reading / per_school_counts * 100

In [15]:
#Creating School Summaries and formatting
per_school_summary_df = pd.DataFrame({
             "School Type": per_school_types,
             "Total Students": per_school_counts,
             "Total School Budget": per_school_budget,
             "Per Student Budget": per_school_capita,
             "Average Math Score": per_school_math,
           "Average Reading Score": per_school_reading,
           "% Passing Math": per_school_passing_math,
           "% Passing Reading": per_school_passing_reading,
           "% Overall Passing": per_overall_passing_percentage})

per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,}".format)

per_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928",$628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356",$582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411",$639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916",$644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500",$625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020",$652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087",$581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635",$655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650",$650.0,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858",$609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [16]:
#Sorting by the best and worst schools
top_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)
bottom_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=True)

In [17]:
#Showing top 5 schools
top_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356",$582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Griffin High School,Charter,1468,"$917,500",$625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574",$578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858",$609.0,83.839917,84.044699,94.594595,95.945946,90.540541
Wright High School,Charter,1800,"$1,049,400",$583.0,83.682222,83.955000,93.333333,96.611111,90.333333
Shelton High School,Charter,1761,"$1,056,600",$600.0,83.359455,83.725724,93.867121,95.854628,89.892107
Holden High School,Charter,427,"$248,087",$581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Thomas High School,Charter,1635,"$1,043,130",$638.0,83.350937,83.896082,66.911315,69.663609,65.076453
Bailey High School,District,4976,"$3,124,928",$628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Ford High School,District,2739,"$1,763,916",$644.0,77.102592,80.746258,68.309602,79.299014,54.289887


In [18]:
#Showing worst 5 schools
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363",$637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411",$639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635",$655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020",$652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650",$650.0,77.072464,80.966394,66.057551,81.222432,53.539172
Ford High School,District,2739,"$1,763,916",$644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Bailey High School,District,4976,"$3,124,928",$628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Thomas High School,Charter,1635,"$1,043,130",$638.0,83.350937,83.896082,66.911315,69.663609,65.076453
Holden High School,Charter,427,"$248,087",$581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Shelton High School,Charter,1761,"$1,056,600",$600.0,83.359455,83.725724,93.867121,95.854628,89.892107


In [19]:
#Create DF by grades
ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]
tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]
eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]
twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

# Group each school Series by the school name for the average math score.
ninth_grade_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_grade_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

# Group each school Series by the school name for the average reading score.
ninth_grade_reading_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_grade_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_grade_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_grade_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

In [31]:
#Getting average in math for 9th grade
ninth_grade_math_scores.mean()

80.12030207173648

In [32]:
#Getting average in reading for 9th grade
ninth_grade_reading_scores.mean()

82.42649372773728

In [20]:
# Combine each Series for average math and reading scores by school into a DataFrame (one for math and other for reading)
math_scores_by_grade = pd.DataFrame({
               "9th": ninth_grade_math_scores,
               "10th": tenth_grade_math_scores,
               "11th": eleventh_grade_math_scores,
               "12th": twelfth_grade_math_scores})

reading_scores_by_grade = pd.DataFrame({
              "9th": ninth_grade_reading_scores,
              "10th": tenth_grade_reading_scores,
              "11th": eleventh_grade_reading_scores,
              "12th": twelfth_grade_reading_scores})

In [21]:
# Formatting each grade column and removing index name
math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:.1f}".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:.1f}".format)
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:.1f}".format)
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:.1f}".format)
math_scores_by_grade.index.name = None

reading_scores_by_grade["9th"] = reading_scores_by_grade["9th"].map("{:,.1f}".format)
reading_scores_by_grade["10th"] = reading_scores_by_grade["10th"].map("{:,.1f}".format)
reading_scores_by_grade["11th"] = reading_scores_by_grade["11th"].map("{:,.1f}".format)
reading_scores_by_grade["12th"] = reading_scores_by_grade["12th"].map("{:,.1f}".format)
reading_scores_by_grade.index.name = None


In [22]:
#Creating and assigning ranges with labels FOR SCHOOL SPENDING
spending_bins =[0,585,630,645,675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

# Calculate averages for the desired columns.
spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [23]:
# Assemble into DataFrame and formatting
spending_summary_df = pd.DataFrame({
          "Average Math Score" : spending_math_scores,
          "Average Reading Score": spending_reading_scores,
          "% Passing Math": spending_passing_math,
          "% Passing Reading": spending_passing_reading,
          "% Overall Passing": overall_passing_spending})
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)
spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)
spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.1f}".format)
spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.1f}".format)
spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.1f}".format)

spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.5,83.9,93.5,96.6,90.4
$585-629,81.9,83.2,87.1,92.7,81.4
$630-644,78.5,81.6,66.9,77.5,56.4
$645-675,77.0,81.0,66.2,81.1,53.5


In [24]:
#Creating and assigning ranges with labels FOR SCHOOL SIZE
size_bins = [0, 1000, 2000, 5000]
group_names_size = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
per_school_counts.groupby(pd.cut(per_school_counts, size_bins, labels=group_names_size)).count()
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels=group_names_size)

# Grupoing by School Size, creating DataFrame and formatting.
size_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

size_summary_df = pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": size_overall_passing})

size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)
size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)
size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.1f}".format)
size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.1f}".format)
size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.1f}".format)

size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.8,83.9,93.6,96.1,89.9
Medium (1000-2000),83.4,83.9,88.3,91.3,85.4
Large (2000-5000),77.7,81.3,70.0,82.8,58.3


In [25]:
#Grouping by School Type, creating DataFrame and formatting
type_math_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]
type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

type_summary_df = pd.DataFrame({
          "Average Math Score" : type_math_scores,
          "Average Reading Score": type_reading_scores,
          "% Passing Math": type_passing_math,
          "% Passing Reading": type_passing_reading,
          "% Overall Passing": type_overall_passing})

type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)
type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)
type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].map("{:.0f}".format)
type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].map("{:.0f}".format)
type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].map("{:.0f}".format)

type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,90,93,87
District,77.0,81.0,67,81,54


In [33]:
per_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size
Bailey High School,District,4976,"$3,124,928",$628.0,77.048432,81.033963,66.680064,81.933280,54.642283,$585-629,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356",$582.0,83.061895,83.975780,94.133477,97.039828,91.334769,<$584,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411",$639.0,76.711767,81.158020,65.988471,80.739234,53.204476,$630-644,Large (2000-5000)
Ford High School,District,2739,"$1,763,916",$644.0,77.102592,80.746258,68.309602,79.299014,54.289887,$630-644,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500",$625.0,83.351499,83.816757,93.392371,97.138965,90.599455,$585-629,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020",$652.0,77.289752,80.934412,66.752967,80.862999,53.527508,$645-675,Large (2000-5000)
Holden High School,Charter,427,"$248,087",$581.0,83.803279,83.814988,92.505855,96.252927,89.227166,<$584,Small (<1000)
Huang High School,District,2917,"$1,910,635",$655.0,76.629414,81.182722,65.683922,81.316421,53.513884,$645-675,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650",$650.0,77.072464,80.966394,66.057551,81.222432,53.539172,$645-675,Large (2000-5000)
Pena High School,Charter,962,"$585,858",$609.0,83.839917,84.044699,94.594595,95.945946,90.540541,$585-629,Small (<1000)
